In [136]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [137]:
cd /content/drive/MyDrive/CS_Vision

/content/drive/MyDrive/CS_Vision


In [138]:
import torch, time, os, pickle
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os, gzip, torch
import scipy.misc
import imageio
import matplotlib.pyplot as plt
import imageio
from torch.nn import functional as F

In [139]:
class Generator(nn.Module):
    # Network Architecture is exactly same as in infoGAN (https://arxiv.org/abs/1606.03657)
    # Architecture : FC1024_BR-FC7x7x128_BR-(64)4dc2s_BR-(1)4dc2s_S
    def __init__(self, input_dim=100, output_dim=1, input_size=32, class_num=10):
        super(Generator, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.input_size = input_size
        self.class_num = class_num

        self.fc = nn.Sequential(
            nn.Linear(self.input_dim + self.class_num, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Linear(1024, 128 * (self.input_size // 4) * (self.input_size // 4)),
            nn.BatchNorm1d(128 * (self.input_size // 4) * (self.input_size // 4)),
            nn.ReLU(),
        )
        self.deconv = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, self.output_dim, 4, 2, 1),
            nn.Tanh(),
        )
        initialize_weights(self)

    def forward(self, input, label):
        x = torch.cat([input, label], 1)
        x = self.fc(x)
        x = x.view(-1, 128, (self.input_size // 4), (self.input_size // 4))
        x = self.deconv(x)

        return x

In [140]:
def initialize_weights(net):
    for m in net.modules():
        if isinstance(m, nn.Conv2d):
            m.weight.data.normal_(0, 0.02)
            m.bias.data.zero_()
        elif isinstance(m, nn.ConvTranspose2d):
            m.weight.data.normal_(0, 0.02)
            m.bias.data.zero_()
        elif isinstance(m, nn.Linear):
            m.weight.data.normal_(0, 0.02)
            m.bias.data.zero_()

def save_images(images, size, image_path):
    return imsave(images, size, image_path)

def imsave(images, size, path):
    image = np.squeeze(merge(images, size))
    # return scipy.misc.imsave(path, image)
    return imageio.imwrite(path, image)

def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    if (images.shape[3] in (3,4)):
        c = images.shape[3]
        img = np.zeros((h * size[0], w * size[1], c))
        for idx, image in enumerate(images):
            i = idx % size[1]
            j = idx // size[1]
            img[j * h:j * h + h, i * w:i * w + w, :] = image
        return img
    elif images.shape[3]==1:
        img = np.zeros((h * size[0], w * size[1]))
        for idx, image in enumerate(images):
            i = idx % size[1]
            j = idx // size[1]
            img[j * h:j * h + h, i * w:i * w + w] = image[:,:,0]
        return img
    else:
        raise ValueError('in merge(images,size) images parameter ''must have dimensions: HxW or HxWx3 or HxWx4')

In [141]:
## Generator 모델 가져오기
model = Generator(input_dim=100, output_dim=1, input_size=28, class_num=10)
optimizer = optim.Adam(model.parameters(), lr=0.0002, betas=(0.5, 0.999))

checkpoint = torch.load('./models/mnist/CGAN/CGAN_G.pkl')
model.load_state_dict(checkpoint)

model.eval()

batch_size =  100
class_num = 10 
z_dim = 100


sample_y_ = torch.zeros(batch_size, class_num).scatter_(1, torch.randint(0, class_num - 1, (batch_size, 1)).type(torch.LongTensor), 1)
sample_z_ = torch.rand((batch_size, z_dim))
# sample_z_, sample_y_ = sample_z_.cuda(), sample_y_.cuda()
print(type(sample_y_))
print(sample_y_.shape)


samples = model(sample_z_, sample_y_)
print(type(samples))
print(samples.shape)
samples = samples.data.numpy().transpose(0, 2, 3, 1)
# print(type(samples))
# print(samples.shape)
samples = (samples + 1) / 2 
save_images(samples[:10 * 10, :, :, :], [10, 10], './inception/img_cgan/test3.png')

<class 'torch.Tensor'>
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 1, 28, 28])


In [142]:
cd ./inception/

/content/drive/My Drive/CS_Vision/inception


In [143]:
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out
def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

In [144]:
import os,sys
import imageio
import numpy as np
import argparse
import math
import torchvision.transforms as transforms
import torch
import cv2 as cv
import glob as glob
from numpy import clip

from skimage.transform import resize
from google.colab.patches import cv2_imshow

in_path = "./img_cgan/"
out_path = in_path + "crop_cgan/"
input_image_dir = out_path
model_dir = "./mnist_model_10.ckpt"
img_size = 28
batch_size = 34
channel = 1
num_splits = 10

GPUID = 0
os.environ["CUDA_VISIBLE_DEVICES"] = str(GPUID)
print ("PACKAGES LOADED")

def load_images(image_dir):
    images = []
    for fn in os.listdir(image_dir):
        ext = os.path.splitext(fn)[1].lower()
        img_path = os.path.join(image_dir, fn)
        img = imageio.imread(img_path)
        img = resize(img, (28, 28))
        # img = img[:, :, 0]
        # print(np.array(img).shape)
        # calculate per-channel means and standard deviations
        means = img.mean(axis=(0, 1), dtype='float64')
        stds = img.std(axis=(0, 1), dtype='float64')
        # per-channel standardization of pixels
        pixels = (img - means) / stds
        pixels = clip(pixels, -1.0, 1.0)
        images.append(pixels)
    # print(np.array(images).shape)
    return images


def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / np.expand_dims(e_x.sum(axis=1), axis=1)   # only difference


def preds2score(preds, splits=10):
    scores = []
    for i in range(splits):
        part = preds[(i * preds.shape[0] // splits):((i + 1) * preds.shape[0] // splits), :]
        kl = part * (np.log(part) - np.log(np.expand_dims(np.mean(part, 0), 0)))
        kl = np.mean(np.sum(kl, 1))
        scores.append(np.exp(kl))
    return np.mean(scores), np.std(scores)

def get_inception_score(images):
    splits = num_splits
    inps = []
    input_transform = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    for img in images:
        img = img.astype(np.float32)
        inps.append(np.expand_dims(img, 0))
    preds = []
    n_batches = 2
    n_preds = 0

    net = ResNet18()
    net.load_state_dict(torch.load(model_dir))
    print("load model successfully")
    
    for i in range(n_batches):
        sys.stdout.write(".")
        sys.stdout.flush()
        inp = inps[(i * batch_size):min((i + 1) * batch_size, len(inps))]
        inp = np.concatenate(inp, 0)
        inp = np.expand_dims(inp, axis=1)
        # inp = torch.from_numpy(inp).cuda()
        inp = torch.from_numpy(inp)
        outputs = net(inp)
        pred = outputs.data.tolist()
        #pred = softmax(pred)
        preds.append(pred)
        n_preds += outputs.shape[0]
    preds = np.concatenate(preds, 0)
    preds = np.exp(preds) / np.sum(np.exp(preds), 1, keepdims=True)
    mean_, std_ = preds2score(preds, splits)
    return mean_, std_

def crop10x10(in_path, out_path):
    # mnist
    x_cors = [0,28,56,84,112,140,168,196,224,252]
    y_cors = [0,28,56,84,112,140,168,196,224,252]
    img_size = 28
    number_channel = 1

    print(out_path)
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    in_list = glob.glob(in_path + "*.png")
    count = 0
    for img_name in in_list:
        count += 1
        if (number_channel == 1):
            img = cv.imread(img_name, 0)
        else:
            img = cv.imread(img_name, 1)
        for x in x_cors:
            for y in y_cors:
                img_crop = img[x:x + img_size, y:y + img_size]
                # print(img_crop.shape)
                if (number_channel == 1):
                    h, w = img_crop.shape
                else:
                    h, w, c = img_crop.shape
                if (h != img_size) or (w != img_size):
                    continue

                out_name = out_path + str(count) + "_" + str(x) + "_" + str(y) + ".png"
                # print(out_name)
                cv.imwrite(out_name, img_crop)

crop10x10(in_path, out_path)                
images = load_images(input_image_dir)
mean, std = get_inception_score(images)
print('\nInception mean: ', mean)
print('Inception std: ', std)



PACKAGES LOADED
./img_cgan/crop_cgan/
load model successfully
..
Inception mean:  4.474873539847748
Inception std:  0.6989032701713527
